In [ ]:
import pandas as pd
import requests as rqst
from bs4 import BeautifulSoup
import time

Define useragent and url

In [ ]:
baseurl = 'https://www.etsy.com/'
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'

# Set the headers to include the user agent
headers = {
    'User-Agent': user_agent,
    'Accept-Language': 'en-US,en;q=0.9',
}

# Start with the first page
page_num = 1

# Keep track of all the product data
product_data = []

# Keep track of the total time taken
total_time = 0


#Input code

In [ ]:
# while True:
#     print('In which field you want to analyze? Please write:\n - search\n - market\n')
#     analyze_preference = input().lower()
    
#     if analyze_preference == 'search':
#         print('Please provide the search term')
#         search_term=input().replace(' ','+')
#         break
#     elif analyze_preference == 'market':
#         print('Please provide the search term')
#         search_term=input().replace(' ','_')
#         break
#     else:
#         print('Invalid input. Please enter "search" or "market".\n')

# print(search_term)

In [ ]:
search_term='budget planner'
search_term=search_term.replace(' ','+')

For market search https://www.etsy.com/market/

In [ ]:
# while True:
#     # Construct the URL for the current page
#     url = f'https://www.etsy.com/market/{search_term}?ref=pagination&page={page_num}'

#     # Send a request to the page
#     try:
#         start_time = time.time()
#         r = rqst.get(url, headers=headers)
#         r.raise_for_status()
#         end_time = time.time()
#         elapsed_time = end_time - start_time
#         total_time += elapsed_time
#         print(f"Page {page_num} scraped in {elapsed_time:.2f} seconds")
#     except rqst.exceptions.HTTPError as err:
#         if err.response.status_code == 404:
#             #print(f"404 Error: Page {page_num} not found")
#             break
#         else:
#             raise

#     # Parse the HTML content of the page
#     soup = BeautifulSoup(r.content, 'lxml')

#     # Find all the product listings on the page
#     listings = soup.find('ul', {'id': 'search-results'})
#     product_listings = listings.find_all('li')

#     # Extract the required data from each listing and append to the product_data list
#     for listing in product_listings:
#         title_elem = listing.select_one('h2.v2-listing-card__title')
#         title = title_elem.text.strip()
#         price_elem = listing.select_one('span.currency-value')
#         price = price_elem.text.strip()
#         review_elem = listing.select_one('span.wt-nudge-l-3')
#         review = review_elem.text.strip() if review_elem else 'N/A'
#         ad_etsy_seller = bool(listing.select_one('span:-soup-contains("by Etsy seller")'))
#         bestseller = bool(listing.select_one('span:-soup-contains("Bestseller")'))
#         starseller = bool(listing.select_one('p.star-seller-badge-lavender-text-light'))
#         product_url_elem = listing.select_one('a.listing-link')
#         product_url = product_url_elem.get('href') if product_url_elem else 'N/A'


#         # Add the extracted data as a dictionary to the product_data list
#         product_data.append({
#             'StarSeller': starseller,
#             'Title': title,
#             'Price': price,
#             'Review': review,
#             'Etsy_AD': ad_etsy_seller,
#             'BestSeller': bestseller,
#             'URL': product_url
#         })

#     # Increment the page number
#     page_num += 1

In [ ]:
while True:
    # Construct the URL for the current page
    url = f'https://www.etsy.com/search?q={search_term}&ref=pagination&page={page_num}&currency=USD'

    # Send a request to the page
    try:
        start_time = time.time()
        r = rqst.get(url, headers=headers)
        r.raise_for_status()
        end_time = time.time()
        elapsed_time = end_time - start_time
        total_time += elapsed_time
        print(f"Page {page_num} scraped in {elapsed_time:.2f} seconds")
    except rqst.exceptions.HTTPError as err:
        if err.response.status_code == 404:
            #print(f"404 Error: Page {page_num} not found")
            break
        else:
            raise

    # Parse the HTML content of the page
    soup = BeautifulSoup(r.content, 'lxml')

    # Find all the product listings on the page
    page_listings = soup.find_all('div', class_='v2-listing-card')

    # Count the number of listings found on the page
    num_products_per_page = len(page_listings)

    print(f'Number of products scraped: {num_products_per_page}')

    # Extract the required data from each listing and append to the product_data list
    for listing in page_listings:
        # Extract the product title
        currency=listing.find('span', class_='currency-symbol').text.strip()
        title = listing.find('h3', class_='v2-listing-card__title').text.strip()
        price= listing.select_one('span.currency-value').text.strip()
        review_elem = listing.select_one('span.wt-nudge-l-3')
        review = review_elem.text.strip() if review_elem else 'N/A'
        ad_etsy_seller = bool(listing.select_one('span:-soup-contains(" Advertisement by Etsy seller")'))
        bestseller = bool(listing.select_one('span:-soup-contains("Bestseller")'))
        starseller = bool(listing.select_one('p.star-seller-badge-lavender-text-light'))
        product_url = listing.select_one('a.listing-link').get('href')
        # Add the extracted data as a dictionary to the product_data list
        product_data.append({
            'StarSeller': starseller,
            'Title': title,
            'Price': price,
            'Review': review,
            'Etsy_AD': ad_etsy_seller,
            'BestSeller': bestseller,
            'Currency': currency,
            'URL': product_url
        })

    page_num += 1

In [ ]:
# Print the total time taken
print(f"Total time taken: {total_time:.2f} seconds")

In [ ]:
# Create a Pandas DataFrame from the product_data list
df = pd.DataFrame(product_data)

# Print the DataFrame
print(df)

In [ ]:
df

In [ ]:
df.tail()

In [ ]:
# Write the DataFrame to a CSV file
df.to_csv('etsy_products.csv', index=False)

ADS not working, get into the produyct and fetch the review for that product